<body>
    <div>
        <h2 style='font-family: cursive; font-size: 40px;'>Factorizacion Cholesky</h2>
<!--         <img src= alt=middle style="height: 340px"> -->
    </div>
</body>

<h3 style="font-family: 'Courier New'; font-size:25px">Octave</h3>

In [1]:
function [error] = checkSDP(a, m, n)
    error = 0;
    if  m != n
        error = 1;
        disp('Error: La matriz no es cuadrada.');
    end
    if !issymmetric(a)
        error = 1;
        disp('Error: La matriz no es simetrica.');
    else
        for i=1:m
            if det(a(1:i, 1:i)) < 0
                error = 1;
                disp('Error: La matriz no es definida positiva.');
            end
        end
    end
end

function [l] = cholesky_aux(a, m, n)
    l(m,m) = 0;
    l(1,1) = sqrt(a(1, 1));

    for i=2:m
        l(i,1) = a(i, 1)/l(1,1);

        for j=2:i-1
            l_sum = 0;
            
            for k=1:j
                l_sum = l_sum + l(j,k) * l(i,k);
            end
            l(i,j) = (a(i,j) - l_sum)/l(j,j);
        end
        l_pow = 0;
        for k=1:i
            l_pow = l_pow + l(i,k)**2;
        end
        l(i,i) = sqrt(a(i,i) - l_pow);
    end
end

function [x] = backwardSubstitution(a, m, n)
    x_i(m) = [a(m,n)/a(m,m)];
    for i=m:-1:1
        x_sum = 0;
        for j=i+1:m
            x_sum = x_sum + a(i, j) * x_i(j);
        end
        x_i(i) = (a(i, n)-x_sum)/a(i,i);
    end
    x = x_i';
end

function [y] = forwardSubstitution(a, m, n)
    x_i(m) = 0;
    x_i(1) = a(1,n)/a(1,1);
    for i=2:m
        x_sum = 0;
        for j=1:i
            x_sum = x_sum + a(i, j) * x_i(j);
        end
        
        if a(i,i) != 0
            x_i(i) = (a(i, n)-x_sum)/a(i,i);
        else
            x_i(i) = 0;
        end
    end
    y = x_i';
end

function [x] = cholesky(a, b)
    [m,n] = size(a);
    if checkSDP(a, m, n)
        x = 0;
    else
        l = cholesky_aux(a, m, n);
        l_2 = [l b];
        [y] = forwardSubstitution(l_2, m, n + 1);
        l_t = [l' y];
        [x] = backwardSubstitution(l_t, m, n + 1);
    end    
end

a = [25 15 -5 -10; 15 10 1 -7; -5 1 21 4; -10 -7 4 18];
b = [-25; -19; -21; -5];

x = cholesky(a, b)

x =

  -1
  -1
  -1
  -1

